In [45]:
import os
import sys
from datetime import datetime
from typing import Dict, List
import requests
import json
import uuid
# Try to import the Todoist API
try:
    from todoist_api_python.api import TodoistAPI
except ImportError:
    print("Error: todoist_api_python package not installed.")
    print("Install it using: pip install todoist-api-python")
    sys.exit(1)

API_TOKEN="28288df256677eba148d4021fc7563100634a5e1"


In [ ]:
def log(message: str) -> None:
    """Print a timestamped log message."""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {message}")

def list_projects_and_tasks() -> None:
    """List all projects and their tasks from Todoist."""
    api = TodoistAPI(get_api_key())
    
    try:
        # Get all projects
        log("Fetching projects...")
        projects = api.get_projects()
        if not projects:
            log("No projects found in your Todoist account.")
            return
            
        # Get all tasks
        log("Fetching tasks...")
        all_tasks = api.get_tasks()
        
        # Group tasks by project
        tasks_by_project: Dict[str, List] = {}
        for task in all_tasks:
            if task.project_id not in tasks_by_project:
                tasks_by_project[task.project_id] = []
            tasks_by_project[task.project_id].append(task)
        
        # Display projects and their tasks
        print("\n" + "="*80)
        print(f"Found {len(projects)} projects and {len(all_tasks)} tasks")
        print("="*80)
        
        for project in sorted(projects, key=lambda p: p.order):
            is_inbox = getattr(project, 'is_inbox_project', False)
            project_name = f"{project.name} {'(Inbox)' if is_inbox else ''}"
            
            print(f"\n## {project_name}")
            print("-" * (len(project_name) + 3))
            
            if project.id in tasks_by_project:
                project_tasks = tasks_by_project[project.id]
                for i, task in enumerate(sorted(project_tasks, key=lambda t: getattr(t, 'order', 0)), 1):
                    due_date = f"(Due: {task.due.date})" if hasattr(task, 'due') and task.due else ""
                    priority = f"[P{task.priority}]" if task.priority and task.priority != 1 else ""
                    labels = f"Tags: {', '.join(['#'+l for l in task.labels])}" if task.labels else ""
                    
                    print(f"{i}. {task.content} {priority} {due_date}")
                    if labels:
                        print(f"   {labels}")
                    if task.description:
                        # Show first line of description if it exists
                        desc_preview = task.description.split('\n')[0]
                        if len(desc_preview) > 50:
                            desc_preview = desc_preview[:47] + "..."
                        print(f"   Description: {desc_preview}")
                    
                    # Add a small spacer between tasks for readability
                    if i < len(project_tasks):
                        print("")
            else:
                print("No tasks in this project")
        
        print("\n" + "="*80)
        
    except Exception as error:
        log(f"✕ Error: {str(error)}")
        sys.exit(1)

# Helper function for logging
def log(message):
    print(message)


In [24]:
log("Starting Todoist Project and Task Lister")
list_projects_and_tasks()
log("Done!")

Starting Todoist Project and Task Lister
Fetching projects...
Fetching tasks...

Found 5 projects and 13 tasks

## Inbox (Inbox)
----------------
1. [Deedy (@deedydas) on X](https://x.com/deedydas/status/1870286960826503532)  

## Home 🏡 
----------
No tasks in this project

## project-demo 
----------------
1. [Rust-for-Malware-Development/Keyloggers/SetWindowsHookEx at main · Whitecat18/Rust-for-Malware-Development](https://github.com/Whitecat18/Rust-for-Malware-Development/tree/main/Keyloggers/SetWindowsHookEx)  

2. [A demonstration of the WebAuthn specification](https://webauthn.io/)  

3. [AI at Meta (@AIatMeta) on X](https://x.com/AIatMeta/status/1871263652495642908)  

4. [meng shao (@shao__meng) on X](https://x.com/shao__meng/status/1857668048783814989?s=12&t=WxkQzfBjHoLeOIYXkUI12Q)  

5. [Artificial Intelligence Syllabus Discussion and Analysis for NTA UGC NET](https://www.youtube.com/watch?v=uB3i-qV6VdM&list=PLxCzCOWd7aiHGhOHV-nwb0HR5US5GFKFI)  

6. https://services.google.c

In [19]:
# Get all projects
log("Fetching projects...")
projects = api.get_projects()
if not projects:
    log("No projects found in your Todoist account.")
    sys.exit(0)
    
projects

Fetching projects...


[Project(color='charcoal', comment_count=0, id='2345575076', is_favorite=False, is_inbox_project=True, is_shared=False, is_team_inbox=False, can_assign_tasks=None, name='Inbox', order=0, parent_id=None, url='https://app.todoist.com/app/project/6WwrJcg2CXqFjCWm', view_style='list'),
 Project(color='charcoal', comment_count=0, id='2345575084', is_favorite=False, is_inbox_project=False, is_shared=False, is_team_inbox=False, can_assign_tasks=None, name='Home 🏡', order=1, parent_id=None, url='https://app.todoist.com/app/project/6WwrJgQX2cq5qm87', view_style='list'),
 Project(color='charcoal', comment_count=0, id='2345575184', is_favorite=False, is_inbox_project=False, is_shared=False, is_team_inbox=False, can_assign_tasks=None, name='project-demo', order=2, parent_id=None, url='https://app.todoist.com/app/project/6WwrM4xm3V243h2M', view_style='list'),
 Project(color='charcoal', comment_count=0, id='2345575209', is_favorite=False, is_inbox_project=False, is_shared=False, is_team_inbox=False,

In [48]:


def reopen_tasks_for_project(api_token, project_name=None, project_id=None):
    """
    Reopens all completed tasks for a specific project using the Todoist Sync API.
    You can specify either project_name or project_id.
    """
    # Base URL for the Sync API
    sync_url = "https://api.todoist.com/sync/v9/sync"
    headers = {
        "Authorization": f"Bearer {api_token}"
    }
    
    try:
        # Step 1: Get all projects
        print("Fetching projects...")
        response = requests.post(
            sync_url,
            headers=headers,
            data={
                "sync_token": "*",
                "resource_types": '["projects"]'
            }
        )
        
        if response.status_code != 200:
            print(f"Error fetching projects: {response.status_code} - {response.text}")
            return
        
        projects_data = response.json()
        projects = projects_data.get("projects", [])
        
        # Find the target project
        target_project = None
        
        if project_id:
            for project in projects:
                if project["id"] == project_id:
                    target_project = project
                    break
        elif project_name:
            for project in projects:
                if project["name"].lower() == project_name.lower():
                    target_project = project
                    break
        
        if not target_project:
            print(f"No project found with the specified {'name' if project_name else 'ID'}: {project_name or project_id}")
            print("\nAvailable projects:")
            for p in projects:
                is_inbox = p.get('is_inbox_project', False)
                print(f"- {p['name']} {'(Inbox)' if is_inbox else ''} [ID: {p['id']}]")
            return
        
        print(f"Found project: {target_project['name']} [ID: {target_project['id']}]")
        
        # Step 2: Get completed tasks using Sync API's completed/get_all endpoint
        print(f"Fetching completed tasks for project '{target_project['name']}'...")
        
        completed_url = "https://api.todoist.com/sync/v9/completed/get_all"
        completed_response = requests.get(
            completed_url,
            headers=headers,
            params={"project_id": target_project["id"]}
        )
        
        if completed_response.status_code != 200:
            print(f"Error fetching completed tasks: {completed_response.status_code} - {completed_response.text}")
            return
        
        completed_data = completed_response.json()
        completed_items = completed_data.get("items", [])
        
        if not completed_items:
            print(f"No completed tasks found in project '{target_project['name']}'.")
            return
        
        # Display summary
        print("\n" + "="*80)
        print(f"Found {len(completed_items)} completed tasks to reopen in project '{target_project['name']}'")
        print("="*80)
        
        # Step 3: Prepare and send commands to uncomplete tasks
        commands = []
        for item in completed_items:
            commands.append({
                "type": "item_uncomplete",
                "uuid": str(uuid.uuid4()),
                "args": {
                    "id": item["task_id"]
                }
            })
        
        # Split commands into batches of 100 (Todoist limit)
        command_batches = [commands[i:i+100] for i in range(0, len(commands), 100)]
        
        # Process each batch
        total_reopened = 0
        
        for batch_index, batch in enumerate(command_batches, 1):
            print(f"\nProcessing batch {batch_index} of {len(command_batches)}...")
            
            # Execute commands in batch
            uncomplete_response = requests.post(
                sync_url,
                headers=headers,
                data={
                    "sync_token": "*",
                    "commands": json.dumps(batch)
                }
            )
            
            if uncomplete_response.status_code != 200:
                print(f"Error reopening tasks: {uncomplete_response.status_code} - {uncomplete_response.text}")
                return
            
            # Process the response
            uncomplete_result = uncomplete_response.json()
            
            # Display results for this batch
            for i, (command, item) in enumerate(zip(batch, completed_items[total_reopened:total_reopened+len(batch)])):
                command_uuid = command["uuid"]
                status = uncomplete_result.get("sync_status", {}).get(command_uuid)
                
                if status == "ok":
                    total_reopened += 1
                    print(f"{total_reopened}. ✓ Reopened: {item['content']}")
                    print(f"   Completed at: {item['completed_at']}")
                else:
                    print(f"{total_reopened + i + 1}. ✕ Failed to reopen: {item['content']} - Status: {status}")
        
        # Final summary
        print("\n" + "="*80)
        print(f"Successfully reopened {total_reopened} of {len(completed_items)} tasks in project '{target_project['name']}'")
        print("="*80)
        print("Note: Please refresh your Todoist interface to see the reopened tasks.")
        print("="*80)
        
    except Exception as error:
        print(f"✕ Error: {str(error)}")
        return


In [49]:
# Option 1: Reopen by project name
# reopen_tasks_for_project(API_TOKEN, project_name="Capture")

# Option 2: Reopen by project ID
reopen_tasks_for_project(API_TOKEN, project_id="2349633007")

Fetching projects...
Found project: Capture [ID: 2349633007]
Fetching completed tasks for project 'Capture'...

Found 17 completed tasks to reopen in project 'Capture'

Processing batch 1 of 1...
1. ✓ Reopened: [Harrison Chase (@hwchase17) on X](https://x.com/hwchase17/status/1898437865132130574)
   Completed at: 2025-03-20T15:03:23.000000Z
2. ✓ Reopened: [Ivan Velichko (@iximiuz) on X](https://x.com/iximiuz/status/1898402568532881907)
   Completed at: 2025-03-20T15:03:22.000000Z
3. ✓ Reopened: [RAVI KUMAR SAHU (@RAVIKUMARSAHU78) on X](https://x.com/RAVIKUMARSAHU78/status/1898607836504154361)
   Completed at: 2025-03-20T15:03:21.000000Z
4. ✓ Reopened: [Deedy (@deedydas) on X](https://x.com/deedydas/status/1894841573311197239)
   Completed at: 2025-02-27T02:00:09.000000Z
5. ✓ Reopened: [Visual Studio Code (@code) on X](https://x.com/code/status/1894509394986967500)
   Completed at: 2025-02-27T01:59:56.000000Z
6. ✓ Reopened: [AK (@_akhaliq) on X](https://x.com/_akhaliq/status/18945843153